# Placement of the flows on the four available "slots"

For every stock in the model check how many and which flows it has.
- per stock loop through the flows.
- if the stock is a destination in the flow then the flow is an inflow in the stock.
- if the stock is a dource in the flow then the flow is an outflow in the stock.
- check whether there are maximally four in- and outflows.
- place the flows on the up, right, down, left positions of the stock drawing.

In [3]:
# Parse all the elements from the vensim model.
from ElementParser import ElementParser

parser = ElementParser('ASD_V2.mdl')
elements = parser.parse()

In [9]:
from ModelObjects import Element, Stock, ValueElement, Flow, Variable, ValueType, ValueTypeConstant, Source, Sink

In [41]:
for element in elements:
    if Stock == True:
        print(element)

In [38]:
type(elements)


list